# Import

In [1]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/se-net-pretrained-imagenet-weights/* /root/.cache/torch/hub/checkpoints/
import torch as tc 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
import os,sys,cv2
from torch.cuda.amp import autocast
import matplotlib.pyplot as plt
import albumentations as A
!python -m pip install --no-index --find-links=/kaggle/input/pip-download-for-segmentation-models-pytorch segmentation-models-pytorch
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from glob import glob

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Looking in links: /kaggle/input/pip-download-for-segmentation-models-pytorch
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/segmentation_models_pytorch-0.3.3-py3-none-any.whl
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/pretrainedmodels-0.7.4.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/efficientnet_pytorch-0.7.1.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - done
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/timm-0.9.2-py3-none-any.whl (from segmentation-models-pytorch)
Processing /kaggle/input/pip-download-for-segmentation-models-pytorch/munch-4.0.0-py2.py3-none-any.whl (from pretrainedmodels==0.7.4->segmentation-models-pytorch)
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=26016c050d498d54b1a548eff

# config

In [2]:
class CFG:
    # ============== pred target =============
    target_size = 1

    # ============== model CFG =============
    model_name = 'Unet'
    backbone = 'se_resnext101_32x4d'

    in_chans = 5 # 입력 채널 수
    # ============== training CFG =============
    image_size = 256
    input_size=256
    drop_egde_pixel = 0
    tile_size = image_size
    stride = tile_size // 2
    assert stride>drop_egde_pixel # stride가 drop_edge_pixel보다 커야 함

    train_batch_size = 16 # 32
    valid_batch_size = train_batch_size * 2

    epochs = 20  # 에폭 수
    lr = 6e-4 # 학습률

    # ============== fold =============
    valid_id = 1 # 검증 세트 ID


    # ============== augmentation =============
    train_aug_list = [
        A.RandomResizedCrop(
            input_size, input_size, scale=(0.8,1.25)),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ToTensorV2(transpose_mask=True),
    ]
    train_aug = A.Compose(train_aug_list)
    valid_aug_list = [
        ToTensorV2(transpose_mask=True),
    ]
    valid_aug = A.Compose(valid_aug_list)

# Model

In [3]:
class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()
        self.CFG = CFG
        self.encoder = smp.Unet(
            encoder_name=CFG.backbone, 
            encoder_weights=weight,
            in_channels=CFG.in_chans,
            classes=CFG.target_size,
            activation=None,
        )

    def forward(self, image):
        output = self.encoder(image)
        # output = output.squeeze(-1)
        return output[:,0]#.sigmoid()


def build_model(weight="imagenet"):
    from dotenv import load_dotenv
    load_dotenv()

    print('model_name', CFG.model_name)
    print('backbone', CFG.backbone)

    model = CustomModel(CFG, weight)

    return model.cuda()

# Functions

In [4]:
def min_max_normalization(x:tc.Tensor)->tc.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape # 입력 텐서의 형태를 저장
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)  # 입력 텐서의 차원을 하나로 평탄화
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9) # 픽셀값을 0과 1사이로 조정 , 1e-9는 작은 상수
    return x.reshape(shape)

class Data_loader(Dataset):
    def __init__(self,path,s="/images/"):
        self.paths=glob(path+f"{s}*.tif")
        self.paths.sort()
        self.bool=s=="/labels/"
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        # 이미지를 그레이스케일로 읽어옴
        img=tc.from_numpy(img)
        if self.bool:
            img=img.to(tc.bool)
        else:
            img=img.to(tc.uint8)
        return img

def load_data(path,s):
    data_loader=Data_loader(path,s)
    data_loader=DataLoader(data_loader, batch_size=16, num_workers=2) # num_workers : 데이터를 로드하는데 사용할 병렬 작업 수
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    return tc.cat(data,dim=0)



def dice_coef(pred:tc.Tensor,target:tc.Tensor,TH=0.5,epsilon=1e-5):
    if tc.any(pred<0) or tc.any(pred>1):
        pred=pred.sigmoid()
    target = target.unsqueeze(1).to(tc.float32)
    pred = (pred>TH).to(tc.float32)
    inter = (target*pred).sum()
    den = target.sum() + pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    return dice

    
class Kaggld_Dataset(Dataset):
    def __init__(self,x:list,y:list,arg=False):
        super(Dataset,self).__init__()
        self.x=x#list[(C,H,W),...] 이미지 데이터 리스트
        self.y=y#list[(C,H,W),...] 레이블 데이터 리스트
        self.image_size=CFG.image_size
        self.in_chans=CFG.in_chans
        self.arg=arg
        if arg:
            self.transform=CFG.train_aug
        else: 
            self.transform=CFG.valid_aug

    def __len__(self) -> int:
        return sum([y.shape[0]-self.in_chans for y in self.y])
    
    def __getitem__(self,index):
        i=0
        for x in self.x:
            if index>x.shape[0]-self.in_chans: # 입력 이미지의 시퀀스 또는 프레임수보다 주어진 index가 더 크면 이를 빼고 인덱스를 조정
                index-=x.shape[0]-self.in_chans
                i+=1
            else:
                break
        x=self.x[i]
        y=self.y[i]
        
        # 이미지와 레이블의 임의의 부분을 추출
        x_index=np.random.randint(0,x.shape[1]-self.image_size)
        y_index=np.random.randint(0,x.shape[2]-self.image_size)

        x=x[index:index+self.in_chans,x_index:x_index+self.image_size,y_index:y_index+self.image_size].to(tc.float32)
        y=y[index+self.in_chans//2,x_index:x_index+self.image_size,y_index:y_index+self.image_size].to(tc.float32)

        # 데이터 증강 수행
        data = self.transform(image=x.numpy().transpose(1,2,0), mask=y.numpy())
        x = data['image']
        y = data['mask']
        # 훈련 중에 추가적인 랜덤변환 수행
        if self.arg:
            i=np.random.randint(4)
            x=x.rot90(i,dims=(1,2))
            y=y.rot90(i,dims=(0,1))
            for i in range(3):
                if np.random.randint(2):
                    x=x.flip(dims=(i,))
                    if i>=1:
                        y=y.flip(dims=(i-1,))
        return x,y


# Load data 

In [5]:
train_x=[]
train_y=[]

root_path="/kaggle/input/blood-vessel-segmentation/"
paths=glob(root_path+"train/*")
paths.sort()
for i,path in enumerate(paths[2:]):#Because the memory is not enough, so I don't use some data.
    if path=="/kaggle/input/blood-vessel-segmentation/train/kidney_3_dense":
        continue
    x=load_data(path,"/images/")
    print(x.shape)
    y=load_data(path,"/labels/")
    train_x.append(x)
    train_y.append(y)

    #(C,H,W)

    #aug
    train_x.append(x.permute(1,2,0))
    train_y.append(y.permute(1,2,0))
    train_x.append(x.permute(2,0,1))
    train_y.append(y.permute(2,0,1))

val_x=load_data(paths[0],"/images/")
val_y=load_data(paths[0],"/labels/")


100%|██████████| 139/139 [01:00<00:00,  2.31it/s]


torch.Size([2217, 1041, 1511])


100%|██████████| 65/65 [00:44<00:00,  1.47it/s]


torch.Size([1035, 1706, 1510])


100%|██████████| 143/143 [00:21<00:00,  6.52it/s]


In [6]:
model=build_model()
model=DataParallel(model) # 모델 병렬처리

model_name Unet
backbone se_resnext101_32x4d


# Training

In [7]:
train_dataset=Kaggld_Dataset(train_x,train_y,arg=True)
train_dataset = DataLoader(train_dataset, batch_size=16, num_workers=2, shuffle=True, pin_memory=True)
val_dataset=Kaggld_Dataset([val_x],[val_y])
val_dataset = DataLoader(val_dataset, batch_size=16, num_workers=2, shuffle=False, pin_memory=True)

model=build_model()
model=DataParallel(model)

loss_fn=nn.BCEWithLogitsLoss()
optimizer=tc.optim.AdamW(model.parameters(),lr=CFG.lr)
scaler=tc.cuda.amp.GradScaler()
scheduler = tc.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CFG.lr,
                                                steps_per_epoch=len(train_dataset), epochs=CFG.epochs+1,
                                                pct_start=0.1,) # 학습률 스케줄링
for epoch in range(CFG.epochs):
    time=tqdm(range(len(train_dataset)))
    losss=0
    scores=0
    a=[]
    b=[]
    c=[]
    d=[]
    for i,(x,y) in enumerate(train_dataset):
        x=x.cuda()
        y=y.cuda()
        x=min_max_normalization(x)

        with autocast(): # 연산 16비트 부동소수점형태
            pred=model(x)
            loss=loss_fn(pred,y)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        scheduler.step()
        score=dice_coef(pred.detach(),y) # dice coefficient 계산
        losss=(losss*i+loss.item())/(i+1) # 현재까지 평균 손실 업데이트
        scores=(scores*i+score)/(i+1) # 현재까지 평균 성능 지표 업데이트
        a.append(losss)
        b.append(scores)
        time.set_description(f"epoch:{epoch},loss:{losss:.4f},score:{scores:.4f},lr{optimizer.param_groups[0]['lr']:.4e}")
        time.update()
        del loss,pred
    time.close()
    val_losss=0
    val_scores=0
    time=tqdm(range(len(val_dataset)))
    for i,(x,y) in enumerate(val_dataset):
        x=x.cuda()
        y=y.cuda()
        x=min_max_normalization(x)

        with autocast():
            with tc.no_grad():
                pred=model(x)
                loss=loss_fn(pred,y)
        score=dice_coef(pred.detach(),y)
        val_losss=(val_losss*i+loss.item())/(i+1)
        val_scores=(val_scores*i+score)/(i+1)
        c.append(val_losss)
        d.append(val_scores)
        time.set_description(f"val-->loss:{val_losss:.4f},score:{val_scores:.4f}")
        time.update()

    time.close()
    tc.save(model.state_dict(),f"./{CFG.backbone}_{epoch}_loss{losss:.2f}_score{scores:.2f}_val_loss{val_losss:.2f}_val_score{val_scores:.2f}.pt")

time.close()

model_name Unet
backbone se_resnext101_32x4d


val-->loss:0.0115,score:0.8891: 100%|██████████| 143/143 [00:18<00:00,  7.90it/s]


In [8]:
len(a)

562